In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shishirsh4393","key":"d93c12343966cfd1ced470b693d797ca"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [16]:
!kaggle datasets download -d gti-upm/leapgestrecog
!unzip leapgestrecog.zip -d leapgestrecog_data

Streaming output truncated to the last 5000 lines.
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0001.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0002.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0003.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0004.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0005.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0006.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0007.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0008.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0009.png  
  inflating: leapgestrecog_data/leapgestrecog/leapGestRecog/07/06_index/frame_07_06_0010.png  

In [17]:
!pip install mediapipe opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 29.8 MB/s eta 0:00:00


In [18]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.utils import to_categorical

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

X_gesture, y_gesture = [], []
gesture_labels = {gesture: idx for idx, gesture in enumerate(os.listdir('leapgestrecog_data/leapGestRecog'))}

for gesture, label in gesture_labels.items():
    for person in os.listdir(f'leapgestrecog_data/leapGestRecog/{gesture}'):
        for img_file in os.listdir(f'leapgestrecog_data/leapGestRecog/{gesture}/{person}'):
            img_path = f'leapgestrecog_data/leapGestRecog/{gesture}/{person}/{img_file}'
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            result = hands.process(img_rgb)

            if result.multi_hand_landmarks:
                landmarks = [lm.x for lm in result.multi_hand_landmarks[0].landmark] + \
                            [lm.y for lm in result.multi_hand_landmarks[0].landmark] + \
                            [lm.z for lm in result.multi_hand_landmarks[0].landmark]
                X_gesture.append(landmarks)
                y_gesture.append(label)

X_gesture = np.array(X_gesture)
y_gesture = to_categorical(np.array(y_gesture), num_classes=len(gesture_labels))


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_gesture, y_gesture, test_size=0.2, random_state=42)

model_gesture = Sequential([
    LSTM(64, return_sequences=True, input_shape=(63, 1)),
    Dropout(0.2),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(len(gesture_labels), activation='softmax')
])

model_gesture.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
X_train = X_train.reshape(-1, 63, 1)
X_test = X_test.reshape(-1, 63, 1)

model_gesture.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

model_gesture.save('hand_gesture_model.h5')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.1962 - loss: 2.2016 - val_accuracy: 0.2654 - val_loss: 2.0135
Epoch 2/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.2518 - loss: 2.0285 - val_accuracy: 0.2751 - val_loss: 1.9367
Epoch 3/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.2799 - loss: 1.9549 - val_accuracy: 0.2641 - val_loss: 1.8776
Epoch 4/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.2894 - loss: 1.9059 - val_accuracy: 0.2931 - val_loss: 1.8635
Epoch 5/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.3217 - loss: 1.8386 - val_accuracy: 0.4036 - val_loss: 1.6220
Epoch 6/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.3967 - loss: 1.6420 - val_accuracy: 0.4608 - val_loss: 1.4648
Epoch 7/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4272 - loss: 1.4771 - val_accuracy: 0.5411 - val_loss: 1.2397
Epoch 8/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5563 - loss: 1.1962 - val_accu

In [12]:
# Download the Facial Recognition Dataset
!kaggle datasets download -d apollo2506/facial-recognition-dataset

# Extract the dataset
!unzip facial-recognition-dataset.zip -d facial_recognition_data


Dataset URL: https://www.kaggle.com/datasets/apollo2506/facial-recognition-dataset
License(s): CC0-1.0
facial-recognition-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  facial-recognition-dataset.zip
replace facial_recognition_data/Testing/Testing/Angry/Angry-1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Initialize lists to hold images and labels
images = []
labels = []

# Define the path to the dataset
dataset_path = '/content/facial_recognition_data/Training/Training'

# Loop through each person in the dataset
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    if os.path.isdir(person_path):
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            # Read the image
            image = cv2.imread(image_path)
            if image is not None:
                # Resize the image to 100x100 pixels
                image = cv2.resize(image, (100, 100))
                # Convert the image from BGR to RGB
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # Append the image and label to the lists
                images.append(image)
                labels.append(person_name)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)


In [5]:
print(f"Total Images Loaded: {len(images)}")
print(f"Total Labels Loaded: {len(labels)}")


Total Images Loaded: 28273
Total Labels Loaded: 28273


In [6]:
# Encode the labels as integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert the integer labels to one-hot encoded labels
labels_one_hot = to_categorical(labels_encoded)


In [7]:
# Normalize the images to the range [0, 1]
images = images.astype('float32') / 255.0


In [8]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Input(shape=(100, 100, 3)),  # Explicit Input layer
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
!free -h  # Check RAM
!df -h    # Check Disk


               total        used        free      shared  buff/cache   available
Mem:            12Gi       8.8Gi       195Mi        13Mi       3.7Gi       3.5Gi
Swap:             0B          0B          0B
Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   32G   81G  29% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G     0  5.7G   0% /dev/shm
/dev/root       2.0G  1.2G  820M  59% /usr/sbin/docker-init
tmpfs           6.4G  184K  6.4G   1% /var/colab
/dev/sda1        92G   65G   27G  71% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [14]:
import gc
del images, labels
gc.collect()


0

In [15]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
707/707 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.2657 - loss: 1.7337 - val_accuracy: 0.3662 - val_loss: 1.5621
Epoch 2/20
707/707 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.3796 - loss: 1.5492 - val_accuracy: 0.4497 - val_loss: 1.4220
Epoch 3/20
707/707 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.4370 - loss: 1.4197 - val_accuracy: 0.4799 - val_loss: 1.3320
Epoch 4/20
707/707 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.4695 - loss: 1.3407 - val_accuracy: 0.4912 - val_loss: 1.2944
Epoch 5/20
707/707 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.5001 - loss: 1.2736 - val_accuracy: 0.5126 - val_loss: 1.2663
Epoch 6/20
707/707 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.5264 - loss: 1.2152 - val_accuracy: 0.5105 - val_loss: 1.2532
Epoch 7/20
707/707 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.5484 - loss: 1.1579 - val_accuracy: 0.5316 - val_loss: 1.2243
Epoch 8/20
707/707 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.5718 - loss: 1.1049 - val_a

In [20]:
from tensorflow.keras.models import save_model
import pickle

# Save the trained model
save_model(model_gesture, "hand_gesture_model.h5")

# Save gesture labels (if they need to be loaded later)
gesture_labels = {0: "Fist", 1: "Palm", 2: "Thumb Up", 3: "Thumb Down", 4: "OK"}

with open("gesture_labels.pkl", "wb") as f:
    pickle.dump(gesture_labels, f)

print("Model and labels saved successfully!")


Model and labels saved successfully!


In [21]:
from tensorflow.keras.models import save_model
import pickle

# Save the trained face recognition model
save_model(model, "face_recognition_model.h5")

# Save the label encodings (mapping of user IDs to face encodings)
face_labels = {"user_1": "John Doe", "user_2": "Alice Smith"}  # Example user mappings

with open("face_labels.pkl", "wb") as f:
    pickle.dump(face_labels, f)

print("Face recognition model and labels saved successfully!")


Face recognition model and labels saved successfully!


In [22]:
from tensorflow.keras.models import load_model
import pickle

# Load the saved model
model_gesture = load_model("hand_gesture_model.h5")

# Load the gesture labels
with open("gesture_labels.pkl", "rb") as f:
    gesture_labels = pickle.load(f)

print("Model and labels loaded successfully!")


Model and labels loaded successfully!


In [23]:
from tensorflow.keras.models import load_model
import pickle

# Load the saved model
model = load_model("face_recognition_model.h5")

# Load the face labels
with open("face_labels.pkl", "rb") as f:
    face_labels = pickle.load(f)

print("Face recognition model and labels loaded successfully!")


Face recognition model and labels loaded successfully!


In [24]:
from tensorflow.keras.models import load_model

# Load the model
model_gesture = load_model("/content/hand_gesture_model.h5")

# Explicitly compile the model
model_gesture.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

print("Model loaded and compiled successfully!")


Model loaded and compiled successfully!


In [25]:
import cv2
import os

def preprocess_image(img_path):
    # Check if file exists before loading
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Error: The file '{img_path}' does not exist.")

    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Error: Failed to load image '{img_path}'. Check file path and format.")

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img_rgb

# Example usage
test_image = "/content/leapgestrecog_data/leapgestrecog/leapGestRecog/00/04_fist_moved/frame_00_04_0004.png"  # Replace with actual path
try:
    processed_img = preprocess_image(test_image)
    print("Image processed successfully!")
except Exception as e:
    print(e)


Image processed successfully!


In [26]:
def predict_gesture(img_path):
    input_data = preprocess_gesture_image(img_path)
    if input_data is not None:
        prediction = model_gesture.predict(input_data)
        predicted_label = np.argmax(prediction)

        # Check if the predicted label exists in gesture_labels
        if predicted_label in gesture_labels:
            return gesture_labels[predicted_label]
        else:
            return f"Unknown Gesture (Label {predicted_label})"
    else:
        return "No hand detected"


In [27]:
import numpy as np
import cv2
import mediapipe as mp
from tensorflow.keras.models import load_model

# Load the trained model
model_gesture = load_model('hand_gesture_model.h5')

# Load MediaPipe Hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Define the gesture labels (same order as during training)
gesture_labels = {0: "Fist", 1: "Palm", 2: "Thumb Up", 3: "Thumb Down", 4: "OK"}  # Modify according to your dataset

# Function to preprocess the image
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img_rgb)

    if result.multi_hand_landmarks:
        landmarks = [lm.x for lm in result.multi_hand_landmarks[0].landmark] + \
                    [lm.y for lm in result.multi_hand_landmarks[0].landmark] + \
                    [lm.z for lm in result.multi_hand_landmarks[0].landmark]
        return np.array(landmarks).reshape(1, 63, 1)  # Reshape for LSTM input
    return None

# Function to predict gesture
def predict_gesture(img_path):
    input_data = preprocess_image(img_path)
    if input_data is not None:
        prediction = model_gesture.predict(input_data)
        predicted_label = np.argmax(prediction)
        return gesture_labels[predicted_label]
    else:
        return "No hand detected"

# Example test
test_image = "/content/leapgestrecog_data/leapGestRecog/07/07_ok/frame_07_07_0006.png"  # Replace with actual test image path
print("Predicted Gesture:", predict_gesture(test_image))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Predicted Gesture: Fist


In [46]:
import cv2
import numpy as np
import pickle
import os
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Load the saved face recognition model
model = load_model("face_recognition_model.h5")
print("Face recognition model loaded successfully!")

# Load the label encodings
with open("face_labels.pkl", "rb") as f:
    face_labels = pickle.load(f)

# Ensure label encoder is properly restored
label_encoder = LabelEncoder()
label_encoder.classes_ = np.array(list(face_labels.keys()))  # Restore class labels

print("Loaded labels:", label_encoder.classes_)  # Debugging statement

def preprocess_face_image(image_path):
    """Preprocesses an image for face recognition."""
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not read image at {image_path}")
            return None

        img = cv2.resize(img, (100, 100))  # Resize to model's expected input
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype('float32') / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=0)  # Expand dimensions to match model input
        return img

    except Exception as e:
        print(f"An error occurred during preprocessing: {e}")
        return None

def recognize_face(image_path):
    """Predicts the person in the given image."""
    preprocessed_image = preprocess_face_image(image_path)
    if preprocessed_image is None:
        return "Error processing image"

    prediction = model.predict(preprocessed_image)

    # Print prediction details for debugging
    print("Model raw prediction:", prediction)

    predicted_label = np.argmax(prediction)
    print("Predicted label index:", predicted_label)

    # Check if the predicted label is within the label encoder's known classes
    if predicted_label >= len(label_encoder.classes_):
        return "Unknown Person"  # The model predicted an unseen label

    predicted_user_id = label_encoder.inverse_transform([predicted_label])[0]

    if predicted_user_id in face_labels:
        predicted_user_name = face_labels[predicted_user_id]
        return predicted_user_name
    else:
        return "Unknown Person"  # User ID not found in dictionary

# Example usage
test_image_path = "/content/facial_recognition_data/Testing/Testing/Sad/Sad-1.jpg"  # Replace with actual test image path
predicted_name = recognize_face(test_image_path)
print(f"Predicted Person: {predicted_name}")


Face recognition model loaded successfully!
Loaded labels: ['user_1' 'user_2']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step
Model raw prediction: [[0.5946667  0.2976092  0.00267218 0.07572567 0.01813317 0.01119297]]
Predicted label index: 0
Predicted Person: John Doe
